In [1]:
from torch import nn, sqrt
import torch
import sys
from math import sqrt

import sys
sys.path.append('../')

from CoAtNet import CoAtNet

In [2]:
import easydict 
args = easydict.EasyDict({ "batch_size": 32, 
                          "epochs": 30, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                          'workers':32,
                         'print_freq':2000,
                         'saved_dir':'../trained_models/checkpoint.pt'})

In [3]:
# x=torch.randn(2,3,224,224)
# model=CoAtNet(3,224)
# out = model(x)
# print(out.shape)

In [4]:
# Data loading code
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets

path = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(path, 'train')
valdir = os.path.join(path, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

In [5]:
# train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
train_sampler = None
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=(train_sampler is None),
    num_workers=4, pin_memory=False, sampler=train_sampler)

In [6]:
# (train_sampler is None)

In [7]:
images, labels = next(iter(train_loader))
images = images.cuda()
labels = labels.cuda()

In [8]:
model=CoAtNet(3,224).cuda()
out = model(images)

initialized
conv2d
conv2d
conv2d
conv2d
conv2d
norm
conv2d
conv2d
conv2d
norm
conv2d
conv2d
conv2d
norm
conv2d
conv2d
conv2d
norm
conv2d
conv2d
Linear
Linear
Linear
Linear
Linear
Linear
Linear
Linear
Linear
Linear
Linear
Linear
Linear


RuntimeError: Given groups=64, weight of size [64, 1, 3, 3], expected input[32, 3, 114, 114] to have 64 channels, but got 3 channels instead

In [ ]:
# labels

In [ ]:
print(out.shape)

In [ ]:
torch.argmax(out, dim = -1)

In [ ]:
args.weight_decay

In [ ]:
# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)
# optimizer = torch.optim.SGD(model.parameters(), 0.1)

In [ ]:
output = model(images)
loss = criterion(output, labels)

In [ ]:
loss

In [ ]:
# model.s4.fc_q.weight.shape

In [ ]:
# model.s4.fc_q.weight[0, :10]
param = model.fc.weight.detach().clone()
param

In [ ]:
# loss.backward()k

In [ ]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [ ]:
model.fc.weight.grad[0,:10]

In [ ]:
model.fc.weight[0,:10]

In [ ]:
param == model.fc.weight

In [ ]:
for i, (images, target) in enumerate(train_loader):
    # measure data loading time
#     data_time.update(time.time() - end)

    if args.gpu is not None:
        images = images.cuda(args.gpu, non_blocking=True)
    if torch.cuda.is_available():
        target = target.cuda(args.gpu, non_blocking=True)

    # compute output
    output = model(images)
    loss = criterion(output, target)
    if i % 100 == 20 :
#         print(model.fc.weight.grad[0,:10])
        print(loss.item())
        print(model.fc.weight[0,:10])

    # compute gradient and do SGD step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()